## Q2.

In a petrochemical industry, the separation of compounds is an important task
especially propane and propene (also known as propylene). At a particular section of
the industry, for instance, there is need to isolate three compounds namely, propane (P),
butane, and propylene (PP). The section runs on two distillation columns in order to
achieve a partial separation of these compounds. 150 kg/h of the mixture runs into the
first column, and its molar composition is P=0.5 and PP=0.2. While the feed for the
second column is the bottom of the first one. We know the molar compositions of all
the exiting streams from both the columns. Table 1 shows all the required molar
composition values. 


However, we need to know the molar flow rates of all the streams of both the columns.
Solve the material balance problem using LU decomposition method. Given, the
molecular weight of propane, butane, and propene is 44g/mol, 58g/mol, and 42g/mol,
respectively


**Column 1**  
P = 0.9, PP = 0.08 (top stream) | P = 0.2, PP = 0.2  (bottom stream)

**Column 2** 
P = 0.95, PP = 0.03 (top stream) | P = 0.05, PP = 0.85  (bottom stream)


To find the molar flow rates (in mol/h) of the four output streams:
- x1 = molar flowrate of Top stream of Column 1
- x2 = molar flowrate of Bottom stream of Column 1
- x3 = molar flowrate of Top stream of Column 2
- x4 = molar flowrate of Bottom stream of Column 2



In [1]:
# Molecular weights in g/mol
MW = {
    "P": 44,
    "B": 58,
    "PP": 42
}

# Total feed in kg/h → convert to grams/h
total_feed_gph = 150 * 1000  # g/h

# Feed composition (molar basis)
feed_mole_frac = {
    "P": 0.5,
    "PP": 0.2,
    "B": 1 - 0.5 - 0.2  # = 0.3
}

# Calculate average molecular weight of feed
MW_avg = (
    feed_mole_frac["P"] * MW["P"]
    + feed_mole_frac["B"] * MW["B"]
    + feed_mole_frac["PP"] * MW["PP"]
)

# Total moles in feed (mol/h)
total_moles_feed = total_feed_gph / MW_avg
total_moles_feed


3138.0753138075315

In [2]:
# Let:
# x1 = molar flowrate of Top stream of Column 1
# x2 = molar flowrate of Bottom stream of Column 1
# x3 = molar flowrate of Top stream of Column 2
# x4 = molar flowrate of Bottom stream of Column 2

# Based on component-wise balances:
# P balance:
# 0.5 * F = 0.9*x1 + 0.2*x2
# 0.2*x2 = 0.95*x3 + 0.05*x4

# PP balance:
# 0.2 * F = 0.08*x1 + 0.2*x2
# 0.85*x4 = 0.03*x3 + 0.2*x2

# Rewriting in Ax = b form:

import numpy as np

A = np.array([
    [0.9, 0.2, 0,   0],
    [0,   0.2, 0.95, 0.05],
    [0.08, 0.2, 0,   0],
    [0,   0.2, 0.03, -0.85]
])

b = np.array([
    feed_mole_frac["P"] * total_moles_feed,
    0.2 * total_moles_feed,
    feed_mole_frac["PP"] * total_moles_feed,
    0  # Since equation: 0.85*x4 = 0.03*x3 + 0.2*x2
])


In [3]:
def lu_decomposition(matrix_A):
    """
    Returns:
        L: Lower triangular matrix
        U: Upper triangular matrix
        P: Permutation matrix
    """
    number_of_rows = matrix_A.shape[0]
    permutation_matrix = np.eye(number_of_rows)
    lower_matrix = np.zeros_like(matrix_A)
    upper_matrix = matrix_A.copy().astype(float)

    for k in range(number_of_rows):
        # Find index of max value in column k from row k to end
        max_row_index = np.argmax(np.abs(upper_matrix[k:, k])) + k

        # Swap rows in U
        upper_matrix[[k, max_row_index]] = upper_matrix[[max_row_index, k]]

        # Swap rows in permutation matrix
        permutation_matrix[[k, max_row_index]] = permutation_matrix[[max_row_index, k]]

        # Swap rows in lower matrix (only for the part that was filled)
        lower_matrix[[k, max_row_index], :k] = lower_matrix[[max_row_index, k], :k]

        # Elimination
        for i in range(k+1, number_of_rows):
            multiplier = upper_matrix[i, k] / upper_matrix[k, k]
            lower_matrix[i, k] = multiplier
            upper_matrix[i, :] = upper_matrix[i, :] - multiplier * upper_matrix[k, :]

    # Fill the diagonal of L with 1s
    np.fill_diagonal(lower_matrix, 1.0)

    return lower_matrix, upper_matrix, permutation_matrix


In [4]:
def forward_substitution(lower_matrix, vector_b):
    """
    Solves Lz = b using forward substitution.
    Returns the intermediate vector z.
    """
    number_of_rows = lower_matrix.shape[0]
    vector_z = np.zeros_like(vector_b)

    for i in range(number_of_rows):
        sum_terms = sum(lower_matrix[i, j] * vector_z[j] for j in range(i))
        vector_z[i] = (vector_b[i] - sum_terms) / lower_matrix[i, i]

    return vector_z


In [5]:
def backward_substitution(upper_matrix, vector_z):
    """
    Solves Ux = z using backward substitution.
    Returns the final solution vector x.
    """
    number_of_rows = upper_matrix.shape[0]
    vector_x = np.zeros_like(vector_z)

    for i in reversed(range(number_of_rows)):
        sum_terms = sum(upper_matrix[i, j] * vector_x[j] for j in range(i + 1, number_of_rows))
        vector_x[i] = (vector_z[i] - sum_terms) / upper_matrix[i, i]

    return vector_x


In [6]:
def solve_using_lu(matrix_A, vector_b):
    """
    Solves the system Ax = b using LU decomposition,
    followed by forward and backward substitution.
    """
    lower_matrix, upper_matrix, permutation_matrix = lu_decomposition(matrix_A)
    print("Lower Matrix (L):\n", lower_matrix)
    print("Upper Matrix (U):\n", upper_matrix)
    # Apply permutation to b
    modified_vector_b = np.dot(permutation_matrix, vector_b)

    # Solve L * z = Pb using forward substitution
    vector_z = forward_substitution(lower_matrix, modified_vector_b)

    # Solve U * x = z using backward substitution
    solution_vector_x = backward_substitution(upper_matrix, vector_z)

    return solution_vector_x


#### Molar flow rates are:

In [ ]:
flow_rates = solve_using_lu(A, b)
print("Molar flow rates of components:", flow_rates)

Lower Matrix (L):
 [[1.         0.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.         1.         1.         0.        ]
 [0.08888889 0.91111111 0.94082126 1.        ]]
Upper Matrix (U):
 [[ 0.9         0.2         0.          0.        ]
 [ 0.          0.2         0.95        0.05      ]
 [ 0.          0.         -0.92       -0.9       ]
 [ 0.          0.          0.          0.80118357]]
Molar flow rates of components: [1148.07633432 2678.84478008   63.38781574  632.55363587]
